# 📳 Earthquake — Seismic Anomaly Detection

This notebook loads the raw accelerometer data from `earthquake_data.csv`, performs EDA, trains an **Isolation Forest** for anomaly (earthquake) detection, and exports the model.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
import joblib, os, warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
print('Libraries loaded ✅')

## 1 — Load & Clean Data

In [ ]:
df = pd.read_csv('../raw/earthquake_data.csv')
print(f'Raw rows: {len(df)}')
df.head()

In [ ]:
# Filter out missing or dropped sensor readings (0,0,0)
df = df[(df['X'] != 0) | (df['Y'] != 0) | (df['Z'] != 0)].copy()

# Calculate magnitude vector
df['Magnitude'] = np.sqrt(df['X']**2 + df['Y']**2 + df['Z']**2)

# Create dummy temporal index assuming 10Hz sampling rate
df.index = pd.date_range(start='2024-01-01', periods=len(df), freq='100ms')
print(f'Clean rows: {len(df)}')
df[['X', 'Y', 'Z', 'Magnitude']].describe()

## 2 — Exploratory Data Analysis

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Time-series Magnitude
axes[0, 0].plot(df.index, df['Magnitude'], color='#8b4513', linewidth=0.8)
axes[0, 0].set_title('Seismic Magnitude Over Time')
axes[0, 0].set_ylabel('m/s^2')

# Distribution
axes[0, 1].hist(df['Magnitude'], bins=30, color='#8b4513', edgecolor='white', alpha=0.8)
axes[0, 1].set_title('Magnitude Distribution')
axes[0, 1].set_xlabel('m/s^2')

# X, Y, Z Time-series
axes[1, 0].plot(df.index, df['X'], alpha=0.7, label='X')
axes[1, 0].plot(df.index, df['Y'], alpha=0.7, label='Y')
axes[1, 0].plot(df.index, df['Z'], alpha=0.7, label='Z')
axes[1, 0].set_title('Raw Accelerometer Data')
axes[1, 0].legend()

# Rolling stats
rolling = df['Magnitude'].rolling(window=100)
axes[1, 1].plot(df.index, df['Magnitude'], alpha=0.4, label='Raw Magnitude')
axes[1, 1].plot(df.index, rolling.mean(), color='red', label='Rolling Mean (10s)')
axes[1, 1].set_title('Rolling Statistics')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 3 — Feature Engineering

In [ ]:
df['mag_rolling_mean'] = df['Magnitude'].rolling(window=50, min_periods=1).mean()
df['mag_rolling_std'] = df['Magnitude'].rolling(window=50, min_periods=1).std().fillna(0)

# Compute acceleration delta (jerk)
df['mag_delta'] = df['Magnitude'].diff().fillna(0)

features = ['Magnitude', 'mag_rolling_mean', 'mag_rolling_std', 'mag_delta']
print(f'Feature-engineered rows: {len(df)}')
df[features].head()

## 4 — Anomaly Detection (Isolation Forest)

In [ ]:
iso_model = IsolationForest(contamination=0.03, random_state=42, n_estimators=100)
df['anomaly'] = iso_model.fit_predict(df[features])
anomalies = df[df['anomaly'] == -1]

plt.figure(figsize=(14, 4))
plt.plot(df.index, df['Magnitude'], label='Normal', color='#d2691e', linewidth=0.8)
plt.scatter(anomalies.index, anomalies['Magnitude'], color='red', s=40, zorder=5, label=f'Tremor Detected ({len(anomalies)})')
plt.title('Earthquake Anomaly Detection (Isolation Forest)')
plt.ylabel('Magnitude')
plt.legend()
plt.tight_layout()
plt.show()

print(f'Anomalies detected: {len(anomalies)} / {len(df)} ({100*len(anomalies)/len(df):.1f}%)')

## 5 — Export Model

In [ ]:
os.makedirs('../models', exist_ok=True)

joblib.dump({
    'isolation_forest': iso_model,
    'features': features,
}, '../models/earthquake_model.pkl')

print('✅ Model exported to models/earthquake_model.pkl')